In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from os.path import join, dirname
from datetime import datetime, timedelta

from bulletin.analysis.ocupacao_leitos import read_excel, create_occupation_table, normalize_workbook, int_float
from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia

pd.set_option('display.max_rows', 500)

In [ ]:
lista_mes = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dec']

In [ ]:
#Usar 0 para hoje e 1 para ontem
algarismo = 1
data = datetime.today() - timedelta(algarismo)
mes = lista_mes[data.month-1]
dia = data.day

# filename = f"ocupação dos leitos por suspeitos confirmados covid 19 no Paraná atualizado em {str(dia).zfill(2)} {mes}.xls"
filename = f"ocupacao.xls"
book = read_excel(join(default_input, 'ocupacao_leitos', filename))

In [ ]:
# for sheet in range(len(book.sheet_names())-8, len(book.sheet_names())):
#      create_occupation_table(book, sheet)

normalize_workbook(book, len(book.sheet_names())-1)

In [ ]:
df = pd.read_excel(join(default_input, 'ocupacao_leitos','destination.xlsx'),skiprows=5,index_col=[0,1, 2, 3],header=[0,1]).fillna(0)
df

In [ ]:
# for column in df.columns:
#     df[column] = df[column].apply(int_float)

In [ ]:
df.columns = df.columns.droplevel(0)
df

In [ ]:
df.index = df.index.set_names(['macrorregiao', 'rs', 'municipio', 'hospital'])
df = df.rename(columns={'Unnamed: 4_level_1':'leito'})
df = df.reset_index()
df

In [ ]:
df['rs'] = df['rs'].str.replace('ª','')
df['rs'] = df['rs'].str.replace('º','').astype(int)

In [ ]:
df

In [ ]:
df.to_excel(join(default_output, 'ocupacao_leitos','taxa_ocupacao_leitos.xlsx'),index=False)
uti = df.loc[df['leito']=='UTI'].copy()
enf = df.loc[df['leito']=='ENF'].copy()

In [ ]:
uti.groupby('rs')[[x for x in range(1,dia+1)]].mean().sort_values('rs').to_excel(join(default_output, 'ocupacao_leitos','teste_taxa_media_ocupacao_leitos_uti.xlsx'))

In [ ]:
for x in range(1,dia+1):
    uti[x] = uti[x].add(1)
    enf[x] = enf[x].add(1)

In [ ]:
uticumperc = uti.groupby('rs')[[x for x in range(1,dia+1)]].prod().sort_values('rs')
enfcumperc = enf.groupby('rs')[[x for x in range(1,dia+1)]].prod().sort_values('rs')
for x in range(1,dia+1):
    uticumperc[x] = uticumperc[x].sub(1)
    enfcumperc[x] = enfcumperc[x].sub(1)

In [ ]:
uticumperc.to_excel(join(default_output, 'ocupacao_leitos','teste_taxa_acumulada_ocupacao_leitos_uti.xlsx'))